# State Chains & System Stress Testing

Demonstrates the `Statics` helper for advanced what-if analysis:
state chain management (push/restore), ZIP load injection,
randomized load Monte-Carlo, and continuation power flow under
varying conditions. Results are mapped geographically to show
where the network is vulnerable.

Import the case and instantiate the `PowerWorld`.

```python
import numpy as np
from esapp import PowerWorld
from esapp.components import *
from examples.statics import Statics

pw = PowerWorld(case_path)
s = Statics(pw)
```

In [ ]:
import numpy as np
from esapp import PowerWorld
from esapp.components import *
from examples.statics import Statics
import ast

with open('../data/case.txt', 'r') as f:
    case_path = ast.literal_eval(f.read().strip())

pw = PowerWorld(case_path)
s = Statics(pw)

In [ ]:
import sys; sys.path.insert(0, '..')
from plot_helpers import (
    plot_state_chain, plot_snapshot_comparison,
    plot_sensitivity_map, plot_voltage_profile,
    plot_pv_curve, plot_histograms,
)

SHAPE = 'US'

## 1. State Chain Basics

A state chain is a rolling buffer of saved PowerWorld case states.
Use `pushstate()` to save the current state and `irestore(n)` to
jump back *n* steps. This enables iterative algorithms that need
to backtrack.

In [ ]:
# Solve base case
V0 = pw.pflow()
v0 = np.abs(V0)
print(f"State 0 (base): V_min = {v0.min():.4f}, V_max = {v0.max():.4f}")

# Initialize a 3-deep state chain and push base case
s.chain(maxstates=3)
s.pushstate(verbose=True)

In [ ]:
# Perturb loads +10%, push state 1
loads = pw.loads()
loads['LoadMW'] *= 1.10
pw[Load, 'LoadMW'] = loads[['BusNum', 'LoadID', 'LoadMW']]
V1 = pw.pflow()
v1 = np.abs(V1)
print(f"State 1 (+10% load): V_min = {v1.min():.4f}")
s.pushstate(verbose=True)

# Perturb again +20% total, push state 2
loads['LoadMW'] *= 1.10 / 1.10  # relative to current
loads = pw.loads()
loads['LoadMW'] *= 1.20
pw[Load, 'LoadMW'] = loads[['BusNum', 'LoadID', 'LoadMW']]
V2 = pw.pflow()
v2 = np.abs(V2)
print(f"State 2 (+20% load): V_min = {v2.min():.4f}")
s.pushstate(verbose=True)

In [ ]:
# Restore back to state 0 (base case, 2 steps back)
s.irestore(n=2, verbose=True)
V_restored = pw.pflow()
print(f"Restored state: max |dV| from base = {np.abs(np.abs(V_restored) - v0).max():.6f}")

In [ ]:
plot_state_chain(
    [v0.values, v1.values, v2.values],
    labels=['Base', '+10% Load', '+20% Load'],
)

## 2. ZIP Load Injection

The `setload()` method injects constant-power (S), constant-current (I),
or constant-impedance (Z) loads at every bus through a special dispatch
load (LoadID='99'). This is useful for modeling different load
characteristics.

In [ ]:
# Restore base case
s.irestore(n=2)
pw.pflow()
v_base = pw.voltage(complex=False, pu=True)[0]

# Inject 5 MW constant-power load at every bus
n_bus = pw.n_bus
s.setload(SP=5.0 * np.ones(n_bus))
pw.pflow()
v_after_sp = pw.voltage(complex=False, pu=True)[0]
print(f"After +5 MW/bus (constant-S): V_min = {v_after_sp.min():.4f}")

# Clear and inject constant-impedance load instead
s.clearloads()
s.setload(ZP=5.0 * np.ones(n_bus))
pw.pflow()
v_after_zp = pw.voltage(complex=False, pu=True)[0]
print(f"After +5 MW/bus (constant-Z): V_min = {v_after_zp.min():.4f}")

# Clean up
s.clearloads()

In [ ]:
plot_snapshot_comparison(v_base.values, v_after_sp.values)

## 3. Monte-Carlo Load Variation

The `randomize_load()` method applies log-normal noise to all bus
loads. Running many samples produces a distribution of voltage
profiles for probabilistic analysis.

In [ ]:
np.random.seed(42)
n_samples = 50
v_min_samples = []
v_max_samples = []

for _ in range(n_samples):
    with pw.snapshot():
        s.randomize_load(scale=1.0, sigma=0.15)
        pw.pflow()
        v = pw.voltage(complex=False, pu=True)[0]
        v_min_samples.append(v.min())
        v_max_samples.append(v.max())

v_min_arr = np.array(v_min_samples)
v_max_arr = np.array(v_max_samples)
print(f"V_min distribution: mean={v_min_arr.mean():.4f}, "
      f"std={v_min_arr.std():.4f}, worst={v_min_arr.min():.4f}")
print(f"V_max distribution: mean={v_max_arr.mean():.4f}, "
      f"std={v_max_arr.std():.4f}, worst={v_max_arr.max():.4f}")
print(f"P(V_min < 0.95) = {(v_min_arr < 0.95).mean():.1%}")

In [ ]:
plot_histograms(
    [v_min_arr, v_max_arr],
    ['Min Voltage per Sample', 'Max Voltage per Sample'],
    ['V_min (pu)', 'V_max (pu)'],
    bins=15,
)

## 4. Voltage Vulnerability Map

Compute per-bus voltage sensitivity by finding the worst-case
voltage depression across the Monte-Carlo samples and mapping it
geographically.

In [ ]:
np.random.seed(42)
all_vmag = []

for _ in range(n_samples):
    with pw.snapshot():
        s.randomize_load(scale=1.0, sigma=0.15)
        pw.pflow()
        v = pw.voltage(complex=False, pu=True)[0]
        all_vmag.append(v.values)

all_vmag = np.array(all_vmag)  # (n_samples, n_bus)
worst_v = all_vmag.min(axis=0)
v_range = all_vmag.max(axis=0) - all_vmag.min(axis=0)

print(f"Buses with worst-case V < 0.95: {(worst_v < 0.95).sum()}")
print(f"Max voltage swing: {v_range.max():.4f} pu")

In [ ]:
# Map voltage swing onto branches (average of endpoint swings)
geo_fields = ['BusNum', 'BusNum:1', 'LineCircuit',
              'Longitude', 'Longitude:1', 'Latitude', 'Latitude:1']
lines_geo = pw[Branch, geo_fields]

bus_nums = pw[Bus, 'BusNum']['BusNum'].to_numpy()
bus_idx = {int(b): i for i, b in enumerate(bus_nums)}
branch_swing = np.array([
    0.5 * (v_range[bus_idx.get(int(r['BusNum']), 0)]
         + v_range[bus_idx.get(int(r['BusNum:1']), 0)])
    for _, r in lines_geo.iterrows()
])

plot_sensitivity_map(
    lines_geo, branch_swing,
    shape=SHAPE,
    title='Voltage Swing Under Random Load (Monte-Carlo)',
    clabel='V swing (pu)',
    cmap='YlOrRd',
    symmetric=False,
)

## 5. Generator Limit Monitoring

Check which generators hit reactive or active power limits under stress.

In [ ]:
# Stress the system by increasing load 15%
with pw.snapshot():
    loads = pw.loads()
    loads['LoadMW'] *= 1.15
    pw[Load, 'LoadMW'] = loads[['BusNum', 'LoadID', 'LoadMW']]
    pw.pflow()

    q_violations = s.gens_above_qmax()
    p_violations = s.gens_above_pmax()

    if q_violations is not None and len(q_violations) > 0:
        print(f"Generators above Q_max: {len(q_violations)}")
        print(q_violations.head())
    else:
        print("No generators above Q_max")

    if p_violations is not None and len(p_violations) > 0:
        print(f"\nGenerators above P_max: {len(p_violations)}")
        print(p_violations.head())
    else:
        print("No generators above P_max")

## 6. Continuation Power Flow

Trace the PV curve by gradually increasing a transfer pattern and
solving power flow at each step. The nose point marks the voltage
stability limit.

In [ ]:
n_buses = len(pw[Bus])
interface = np.ones(n_buses)
interface /= interface.sum()  # 1 MW distributed uniformly

# Track the critical bus (lowest initial voltage)
V_base = pw.pflow()
critical_idx = np.argmin(np.abs(V_base))
print(f"Critical bus index: {critical_idx}, V = {np.abs(V_base[critical_idx]):.4f} pu")

mw_pts, v_pts = [], []
for mw in s.continuation_pf(
    interface=interface,
    step_size=0.05,
    min_step=0.001,
    max_step=0.1,
    maxiter=200,
    verbose=True,
    restore_when_done=True,
):
    V = pw.voltage()
    mw_pts.append(mw)
    v_pts.append(np.abs(V[critical_idx]))

print(f"\nCollected {len(mw_pts)} points")
if mw_pts:
    print(f"Transfer range: {min(mw_pts):.1f} to {max(mw_pts):.1f} MW")

In [ ]:
plot_pv_curve(mw_pts, v_pts)